In [ ]:
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
import keras
import matplotlib.pyplot as plt
import tensorflow as tf 
import cv2
import os


input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

initial_mode = keras.initializers.lecun_normal(seed=None)
initial_mode2 = keras.initializers.lecun_normal(seed=None)

for i in range(1,2):
    ## Build the Convolution autoencoders model

    x = Conv2D(16, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # at this point the representation is (4, 4, 8) i.e. 128-dimensional

    x = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(encoded)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(8, (3, 3), activation='relu', padding='same',kernel_initializer=initial_mode)(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu',kernel_initializer=initial_mode)(x)
    x = UpSampling2D((2, 2))(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same',kernel_initializer=initial_mode2)(x)



    autoencoder = Model(input_img, decoded)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

    from keras.datasets import mnist
    import numpy as np

    (x_train, _), (x_test, _) = mnist.load_data()

    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
    x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

    from keras.callbacks import TensorBoard

    
    

    class CustomSaver(tf.keras.callbacks.Callback):
        def on_epoch_begin(self, epoch, logs={}):            
            loop = epoch + 1
            
            if loop%5 == 0:  # or save after some epoch, each k-th epoch etc.
                print('Save the image')
                decoded_imgs = autoencoder.predict(x_test)
                img = decoded_imgs[0]
                img = img.reshape(28,28)
                
                # save the img
                current_dir = os.getcwd()
                plt.imshow(img,'gray')
                plt.savefig(current_dir+'/Decoded_img/Decoded_image_epoch'+str(loop)+'.png')


    saver = CustomSaver()
    current_dir = os.getcwd()

    result = autoencoder.fit(x_train, x_train,
                                        epochs=50,
                                        batch_size=128,
                                        shuffle=True,
                                        validation_data=(x_test, x_test),
                                        callbacks=[saver,TensorBoard(log_dir=current_dir+'/tensorboard_data/lecun')])
    
    


#print(autoencoder.summary())




#decoded_images = autoencoder.predict(x_test)

In [ ]:
decoded_imgs = autoencoder.predict(x_test)

In [ ]:
b = x_test[0]
b = b.reshape(28,28)

plt.imshow(b,'gray')
#plt.savefig('data_test_src.png')


In [ ]:
decoded_imgs.shape
a = decoded_imgs[0]
a = a.reshape(28,28)
plt.imshow(a,'gray')


In [ ]:
autoencoder.save('my_model.h5')

In [ ]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

current_dir = os.getcwd()
new_model = tf.keras.models.load_model(current_dir+'/model/my_CAE_zeros.h5')

In [ ]:
#print(new_model.summary())

In [ ]:
current_dir = os.getcwd()

test_img = cv2.imread('test1.png',cv2.IMREAD_GRAYSCALE)
test_img.shape


In [ ]:
#Do scale
test_img = abs(255-test_img)
test_img = test_img/255


#change the input shape
new_img = cv2.resize(test_img,(28, 28))
new_img.shape
plt.imshow(new_img,cmap='gray')
#plt.savefig('test_src.png')

new_img1 = new_img.reshape(1,28,28,1) 


In [ ]:
new_img1.shape

In [ ]:
pred = new_model.predict(new_img1)

output = pred.reshape(28,28) 

plt.imshow(output,cmap='gray')
plt.savefig('test_zeros.png')
